In [9]:
import pandas as pd
import numpy as np


# Load the dataset
df = pd.read_csv("su_mens_soccer_2024.csv")

# Total games played
total_games = df['gp'].max()
print("Total games played:", total_games)

# Total players
print("Total players:", len(df))

# Top scorer
top_scorer = df.loc[df['g'].idxmax()]
print(f"Top scorer: {top_scorer['player']} with {top_scorer['g']} goals")

# Most assists
top_assist = df.loc[df['a'].idxmax()]
print(f"Most assists: {top_assist['player']} with {top_assist['a']} assists")

# Most minutes played
most_minutes = df.loc[df['min'].idxmax()]
print(f"Most minutes played: {most_minutes['player']} with {most_minutes['min']} minutes")

# Total team goals
total_goals = df['g'].sum()
print("Total team goals:", total_goals)

# Average goals per player
print("Average goals per player:", round(df['g'].mean(), 2))

# Goal + Assist per 90 minutes (efficiency metric)
df['g+a_per_90'] = (df['g'] + df['a']) / (df['min'] / 90)
df_sorted = df[['player', 'g', 'a', 'min', 'g+a_per_90']].sort_values(by='g+a_per_90', ascending=False)

print("\n Top contributors (G+A per 90 mins):")
print(df_sorted.head())

# Save to CSV for visualizations
df_sorted.to_csv("top_contributors_per_90.csv", index=False)

Total games played: 17
Total players: 23
Top scorer: threadgold, gabe with 4 goals
Most assists: acquah, michael with 6 assists
Most minutes played: hut, tomas with 1460 minutes
Total team goals: 25
Average goals per player: 1.09

 Top contributors (G+A per 90 mins):
              player  g  a   min  g+a_per_90
12    gradus, michal  1  0    19    4.736842
1    acquah, michael  1  6   719    0.876217
16    leibold, giona  0  1   178    0.505618
0   threadgold, gabe  4  3  1268    0.496845
8       widman, ezra  2  0   457    0.393873


## Prompt 1

* How many games did SU Mens team play this season?
  17 chatgpt answered correctly and so was my analysis correct (ChatGPT)

## Prompt 2

* Who scored the most goals in the 2024 season?
  The Syracuse men’s soccer team’s top goal scorer during the 2024 season was Gabe Threadgold, who led the team with 4 goals over 17     matches (ChatGPT)

## Prompt 3 

* How many total goals were scored by the team?
  The Syracuse University (SU) Men’s Soccer team scored a total of 25 goals during the 2024 season. According to the team’s cumulative stats for the season, the “Total” goals tally stands at 25 over 17 games (ChatGPT)

## Prompt 4 Which player had the highest impact per minute played, considering both goals and assists?
Looking at the SU men’s soccer stats for the 2024 season, we can calculate a simple impact-per-minute metric using total points (where goals count as 2 pts and assists as 1 pt, according to their official “pts” column) divided by minutes played.

From the cumulative stats:
	•	Michal Gradus: 2 points in 19 minutes ≈ 0.1053 pts/min (We cannot consider a player who just played for 19 mins as the most impact player) that is where chat gpt fumbled a bit. So after I put in the prompt of more than 10 matches then the output was Michael Acquah
    

In [23]:
# Prompt 4: Highest impact per minute (G+A per 90 mins)
impact_players = df[df['gp'] >= 11]
impact_players = impact_players[impact_players['min'] > 0]
impact_players['g+a_per_90'] = (impact_players['g'] + impact_players['a']) / (impact_players['min'] / 90)

if not impact_players.empty:
    prompt_4_player = impact_players.loc[impact_players['g+a_per_90'].idxmax(), 'player']
    prompt_4_value = impact_players['g+a_per_90'].max()
else:
    prompt_4_player = "None"
    prompt_4_value = 0
print("Prompt 4 - Highest G+A per 90 (Min 11 games played):")
print(f"→ {prompt_6_player} with {round(prompt_6_value, 2)} G+A per 90\n")

Prompt 4 - Highest G+A per 90 (Min 11 games played):
→ acquah, michael with 0.88 G+A per 90



## Prompt 5 Player with the most shots but lowest accuracy
* Name the player with the most shots but low accuracy
  Nicholas Kaloukian name given by (chatGPT) but thats incorrect here chatgpt halucinated. I later added the prompt to get the player with atleast 10 shots or more and accuracy means shots on goal out of total shots and nicholas has 50% of shots on target so thats not correct so the new answer is Elton Chifamba

In [24]:
# Prompt 5: Player with >10 shots and worst sog%
low_sog = df[(df['sh'] > 10)]  # only players with more than 10 shots

if not low_sog.empty:
    worst_player = low_sog.loc[low_sog['sog%'].idxmin()]
    prompt_5_player = worst_player['player']
    prompt_5_shots = worst_player['sh']
    prompt_5_sog_percent = worst_player['sog%']
    
    print("Prompt 5 - Player with >10 shots and worst Shots-on-Goal %:")
    print(f"→ {prompt_5_player} with {prompt_5_shots} shots and {prompt_5_sog_percent*100:.1f}% shots on goal\n")
else:
    print("→ No qualifying players found.\n")

Prompt 5 - Player with >10 shots and worst Shots-on-Goal %:
→ chifamba, elton with 11 shots and 9.1% shots on goal



## Prompt 6 As a manager, what areas should I improve to win the league — attack or defence?
* Asked whether the team needs to improve in **attack** or **defence** to win the league.
  ChatGPT analyzed total shots (233), goals (25), shot accuracy (42.9%), and goal conversion rate (10.7%) from the dataset. Since no data was available on goals conceded, defence couldn't be assessed.
  Based on low conversion, it correctly concluded that the **attack** needs improvement — recommending focus on finishing, better decision-making in the final third, and improving chance quality. This answer is accurate and supported by the data.
  In this case the LLM again failed to give the correct answer it did not take into conideration opponents stats and hence the output was attack but actually when we focus on opponent stats we have to work on our defence.


In [29]:
import pandas as pd

# Load and clean dataset
df = pd.read_csv("su_mens_soccer_2024.csv")
df.columns = df.columns.str.strip().str.lower()

# Exclude summary rows if present
df_clean = df[~df['player'].str.lower().isin(['total', 'opponents'])]

# Team totals
team_goals = df_clean['g'].sum()
team_shots = df_clean['sh'].sum()
team_sog = df_clean['sog'].sum()

# Opponent stats from original data
opp_goals = 25
opp_shots = 204
opp_sog = 82

# Percentages
team_sh_pct = team_goals / team_shots if team_shots else 0
opp_sh_pct = opp_goals / opp_shots if opp_shots else 0
team_sog_pct = team_sog / team_shots if team_shots else 0
opp_sog_pct = opp_sog / opp_shots if opp_shots else 0

# Decision logic
if (team_sog_pct - opp_sog_pct) > (opp_sh_pct - team_sh_pct):
    focus = "defense"
elif (opp_sh_pct - team_sh_pct) > (team_sog_pct - opp_sog_pct):
    focus = "attack"
else:
    focus = "both equally"

# Output
print(f"""
Team Shot %:      {team_sh_pct:.3f}
Opponent Shot %:  {opp_sh_pct:.3f}
Team SOG %:       {team_sog_pct:.3f}
Opponent SOG %:   {opp_sog_pct:.3f}
Focus Area:       {focus}
""")


Team Shot %:      0.107
Opponent Shot %:  0.123
Team SOG %:       0.429
Opponent SOG %:   0.402
Focus Area:       defense

